# Finding our best-performing salespeople and products



## Overview of the data

The data for this case is contained in the [`AdventureWorks.db`](AdventureWorks.db) SQLite database. We will be focusing on the tables that belong to the Sales and Product categories. Complete documentation, with schemas, for the original data (of which you have only a subset) can be found [here](https://dataedo.com/download/AdventureWorks.pdf).

**Product Tables (Pg. 34 in documentation):**
* **Product**: one row per product that the company sells
* **ProductReview**: one row per rating and review left by customers
* **ProductModelProductDescriptionCulture**: a link between products and their longer descriptions also indicating a "culture" - which language and region the product is for
* **ProductDescription**: a longer description of each product, for a specific region
* **ProductCategory**: the broad categories that products fit into
* **ProductSubCategory**: the narrower subcategories that products fit into

**Sales Tables (Pg. 71 in documentation):**
* **SalesPerson**: one row per salesperson, including information on their commission and performance
* **SalesOrderHeader**: one row per sale summarizing the sale
* **SalesOrderDetail**: many rows per sale, detailing each product that forms part of the sale
* **SalesTerritory**: the different territories where products are sold, including performance
* **CountryRegionCurrency**: the currency used by each region
* **CurrencyRate**: the average and closing exchange rates for each currency compared to the USD

**Tip**: Review the documentation carefully to learn more about the tables (like relevant columns in each) and the relationships between them. Note that not all columns may be available in the subset provided in this case as they are not necessary for the following exercises. 

Importing the libraries and the `sql` extension:

In [ ]:
%load_ext sql
import pandas as pd

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Let's now load in the database:

In [ ]:
%sql sqlite:///AdventureWorks.db

Remember that in order to run SQL queries from within this notebook, you should write `%%sql` at the start of your code cell, like this:

In [ ]:
%%sql
SELECT * FROM product LIMIT 5;

 * sqlite:///AdventureWorks.db
Done.


productid,NAME,productnumber,makeflag,finishedgoodsflag,color,safetystocklevel,reorderpoint,standardcost,listprice,size,sizeunitmeasurecode,weightunitmeasurecode,weight,daystomanufacture,productline,class,style,productsubcategoryid,productmodelid,sellstartdate,sellenddate,discontinueddate,rowguid,modifieddate
1,Adjustable Race,AR-5381,f,f,,1000,750,0.0,0.0,,,,,0,,,,,,2008-04-30 00:00:00,,,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
2,Bearing Ball,BA-8327,f,f,,1000,750,0.0,0.0,,,,,0,,,,,,2008-04-30 00:00:00,,,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
3,BB Ball Bearing,BE-2349,t,f,,800,600,0.0,0.0,,,,,1,,,,,,2008-04-30 00:00:00,,,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
4,Headset Ball Bearings,BE-2908,f,f,,800,600,0.0,0.0,,,,,0,,,,,,2008-04-30 00:00:00,,,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
316,Blade,BL-2036,t,f,,800,600,0.0,0.0,,,,,1,,,,,,2008-04-30 00:00:00,,,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


## Finding our most popular products

The company would like to know which of their products is the most popular among customers. You figure that the average rating given in reviews is correlated with the number of sales of a particular product (that products with higher reviews have more sales).

### Exercise 1 (1 point)

Using the `product` and `productreview` tables, `INNER JOIN` them and rank the products according to their average review rating. Save the SQL code in a string variable called `rating_ranking`. Please make *absolutely sure* to name your variable exactly that or otherwise your answer will not be recorded.

Your output should look like this:

| productid 	| NAME 	| avgrating 	| num_ratings 	|
|-:	|-:	|-:	|-	|
| 709 	| Mountain Bike Socks, M 	| 5.0 	| 1 	|
| ... 	| ... 	| ... 	| ... 	|


In [ ]:
rating_ranking = """
SELECT 
    product.productid, product.NAME, AVG(productreview.rating) as avgrating, COUNT(productreview.productid) as num_ratings
FROM 
    product
INNER JOIN 
    productreview ON product.productid = productreview.productid
GROUP BY 
    NAME
ORDER BY 
    avgrating DESC;
"""

In [ ]:
%%sql

SELECT 
    product.productid, product.NAME, AVG(productreview.rating) as avgrating, COUNT(productreview.productid) as num_ratings
FROM 
    product
INNER JOIN 
    productreview ON product.productid = productreview.productid
GROUP BY 
    NAME
ORDER BY 
    avgrating DESC;

 * sqlite:///AdventureWorks.db
Done.


productid,NAME,avgrating,num_ratings
798,"Road-550-W Yellow, 40",5.0,1
709,"Mountain Bike Socks, M",5.0,1
937,HL Mountain Pedal,3.0,2


### Exercise 2 

Much to your disappointment, there are only three products with ratings and only four reviews in total! This is nowhere near enough to perform an analysis of the correlation between reviews and total sales. Since we cannot infer the most popular products from the reviews, we will go with an alternative strategy.

#### 2.1 (1 point)

Get the product model ID and description for each product. Include only descriptions for which `productmodelproductdescriptionculture.cultureid = 'en'`.

Your output should look like this:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>productmodelid</th>      <th>description</th>    </tr>  </thead>  <tbody>    <tr>      <td>1</td>      <td>Light-weight, wind-resistant, packs to fit into a pocket.</td>    </tr>    <tr>      <td>2</td>      <td>Traditional style with a flip-up brim; one-size fits all.</td>    </tr>    <tr>      <td>3</td>      <td>Synthetic palm, flexible knuckles, breathable mesh upper. Worn by the AWC team riders.</td>    </tr>    <tr>      <td>...</td>      <td>...</td>    </tr>  </tbody></table>

In [ ]:
productmodelid_description="""
SELECT 
    pmi.productmodelid, pd.description
FROM 
    ProductModelProductDescriptionCulture AS pmi
INNER JOIN 
    ProductDescription AS pd ON pmi.productdescriptionid = pd.productdescriptionid
WHERE 
    pmi.cultureid ='en'
"""


#### 2.2 (2 points)

Get the model ID, name, description, and total number of sales for each product and display the top-10 selling products. You can infer how often products have been sold by looking at the `salesorderdetail` table (each row might indicate more than one sale, so take note of `OrderQty`).

Your output should look like this:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>productmodelid</th>      <th>description</th>      <th>NAME</th>      <th>total_orders</th>    </tr>  </thead>  <tbody>    <tr>      <td>2</td>      <td>Traditional style with a flip-up brim; one-size fits all.</td>      <td>AWC Logo Cap</td>      <td>8311</td>    </tr>    <tr>      <td>111</td>      <td>AWC logo water bottle - holds 30 oz; leak-proof.</td>      <td>Water Bottle - 30 oz.</td>      <td>6815</td>    </tr>    <tr>      <td>33</td>      <td>Universal fit, well-vented, lightweight , snap-on visor.</td>      <td>Sport-100 Helmet, Blue</td>      <td>6743</td>    </tr>    <tr>      <td>...</td>      <td>...</td>      <td>...</td>      <td>...</td>    </tr>  </tbody></table>

**Hint:** Make the query you wrote in exercise 2.1 a temporary view with the `WITH ... AS` syntax. It will give you the English descriptions of the products as a starting point. Then `INNER JOIN` it with all the other relevant tables.

In [ ]:
description_totalorders= """
WITH 
productmodelid_description(productmodelid, description) AS (
    SELECT 
        pmi.productmodelid, pd.description
    FROM 
        ProductModelProductDescriptionCulture AS pmi
    INNER JOIN 
        ProductDescription AS pd ON pmi.productdescriptionid = pd.productdescriptionid
    WHERE 
        pmi.cultureid ='en')

SELECT 
    pmd.productmodelid, pmd.description, p.name, sum(sd.orderqty) AS total_orders
FROM 
    productmodelid_description AS pmd
INNER JOIN 
    Product AS p on p.productmodelid = pmd.productmodelid
INNER JOIN 
    salesorderdetail AS sd on sd.productid = p.productid
GROUP BY 
    p.name
ORDER BY 
    total_orders desc
LIMIT 10
"""


In [ ]:
%%sql

WITH 
productmodelid_description(productmodelid, description) AS (
    SELECT 
        pmi.productmodelid, pd.description
    FROM 
        ProductModelProductDescriptionCulture AS pmi
    INNER JOIN 
        ProductDescription AS pd ON pmi.productdescriptionid = pd.productdescriptionid
    WHERE 
        pmi.cultureid ='en')

SELECT 
    pmd.productmodelid, pmd.description, p.name, sum(sd.orderqty) AS total_orders
FROM 
    productmodelid_description AS pmd
INNER JOIN 
    Product AS p on p.productmodelid = pmd.productmodelid
INNER JOIN 
    salesorderdetail AS sd on sd.productid = p.productid
GROUP BY 
    p.name
ORDER BY 
    total_orders desc
LIMIT 10

 * sqlite:///AdventureWorks.db
Done.


productmodelid,description,NAME,total_orders
2,Traditional style with a flip-up brim; one-size fits all.,AWC Logo Cap,8311
111,AWC logo water bottle - holds 30 oz; leak-proof.,Water Bottle - 30 oz.,6815
33,"Universal fit, well-vented, lightweight , snap-on visor.","Sport-100 Helmet, Blue",6743
11,Unisex long-sleeve AWC logo microfiber cycling jersey,"Long-Sleeve Logo Jersey, L",6592
33,"Universal fit, well-vented, lightweight , snap-on visor.","Sport-100 Helmet, Black",6532
33,"Universal fit, well-vented, lightweight , snap-on visor.","Sport-100 Helmet, Red",6266
1,"Light-weight, wind-resistant, packs to fit into a pocket.","Classic Vest, S",4247
114,"Includes 8 different size patches, glue and sandpaper.",Patch Kit/8 Patches,3865
32,"Short sleeve classic breathable jersey with superior moisture control, front zipper, and 3 back pockets.","Short-Sleeve Classic Jersey, XL",3864
11,Unisex long-sleeve AWC logo microfiber cycling jersey,"Long-Sleeve Logo Jersey, M",3636


### Exercise 3

To get a better sense of the sales, let's look at the correlation between quantity sold and price for each subcategory.

#### 3.1 (1 point)

Write a query that shows how many items were ordered in total for every product in the database. Do not filter by culture.

Your output should look like this:

| productid 	| quantity 	|
|-:	|-:	|
| 707 	| 6266 	|
| 708 	| 6532 	|
| 709 	| 1107 	|
| 710 	| 90 	|
| 711 	| 6743 	|
| 712 	| 8311 	|
| 713 	| 429 	|
| 714 	| 3636 	|
| ... 	| ... 	|

**Hint:** Use the `salesorderdetail` table.

In [ ]:
quantities_ordered = """
SELECT 
    productid, SUM(orderqty) as quantity
FROM 
    SalesOrderDetail
GROUP BY 
    productid
"""


In [ ]:
%%sql

SELECT 
    productid, SUM(orderqty) as quantity
FROM 
    SalesOrderDetail
GROUP BY 
    productid
LIMIT 10

 * sqlite:///AdventureWorks.db
Done.


productid,quantity
707,6266
708,6532
709,1107
710,90
711,6743
712,8311
713,429
714,3636
715,6592
716,2980


#### 3.2 (1 point)

Write a query that shows the list price for each product, alongside its category and subcategory. Your output should look like this:

| productid 	| category 	| subcategory 	| listprice 	|
|-:	|-:	|-:	|-:	|
| 680 	| Components 	| Road Frames 	| 1431.5 	|
| 706 	| Components 	| Road Frames 	| 1431.5 	|
| 707 	| Accessories 	| Helmets 	| 34.99 	|
| 708 	| Accessories 	| Helmets 	| 34.99 	|
| 709 	| Clothing 	| Socks 	| 9.5 	|
| 710 	| Clothing 	| Socks 	| 9.5 	|
| 711 	| Accessories 	| Helmets 	| 34.99 	|
| 712 	| Clothing 	| Caps 	| 8.99 	|
| 713 	| Clothing 	| Jerseys 	| 49.99 	|
| 714 	| Clothing 	| Jerseys 	| 49.99 	|
| 715 	| Clothing 	| Jerseys 	| 49.99 	|
| 716 	| Clothing 	| Jerseys 	| 49.99 	|
| 717 	| Components 	| Road Frames 	| 1431.5 	|
| 718 	| Components 	| Road Frames 	| 1431.5 	|
| 719 	| Components 	| Road Frames 	| 1431.5 	|
| ... 	| ... 	| ... 	| ... 	|

**Hint:** You will find the product categories in the `productcategory` table, and the subcategories in the `productsubcategory` table.

In [ ]:
products_prices="""
SELECT 
    p.productID, pc.name AS category, psc.name AS subcategory, p.listprice
FROM 
    product AS p
JOIN 
    productsubcategory AS psc ON psc.productsubcategoryID = p.productsubcategoryID
JOIN 
    productcategory AS pc ON psc.productcategoryID = pc.productcategoryID
GROUP BY 
    p.productID
"""


#### 3.3 (3 points)

Merge the queries from exercises 3.1 and 3.2 to obtain a table that shows, for each subcategory, the average list price and the total quantity of products sold. Your output should look like this:

| category 	| subcategory 	| average_price_in_subcategory 	| total_items_sold_in_subcategory 	|
|-:	|-:	|-:	|-:	|
| Accessories 	| Bike Racks 	| 120.0 	| 3166 	|
| Accessories 	| Bike Stands 	| 159.0 	| 249 	|
| Accessories 	| Bottles and Cages 	| 7.989999999999999 	| 10552 	|
| Accessories 	| Cleaners 	| 7.95 	| 3319 	|
| Accessories 	| Fenders 	| 21.98 	| 2121 	|
| Accessories 	| Helmets 	| 34.99 	| 19541 	|
| Accessories 	| Hydration Packs 	| 54.99 	| 2761 	|
| Accessories 	| Locks 	| 25.0 	| 1087 	|
| Accessories 	| Pumps 	| 19.99 	| 1130 	|
| Accessories 	| Tires and Tubes 	| 19.482727272727274 	| 18006 	|
| Bikes 	| Mountain Bikes 	| 1683.3649999999982 	| 28321 	|
| Bikes 	| Road Bikes 	| 1597.45 	| 47196 	|
| Bikes 	| Touring Bikes 	| 1425.2481818181814 	| 14751 	|
| Clothing 	| Bib-Shorts 	| 89.99 	| 3125 	|
| ... 	| ... 	| ... 	| ... 	|

**Hint:** To have two `WITH ... AS` statements in the same query, you separate the subqueries with a comma and don't write `WITH` again. Like this:

~~~sql
WITH first_query_alias AS
(
    SELECT ...
),
second_query_alias AS -- Notice we didn't include a second WITH here
(
    SELECT...
)
SELECT ...
~~~

In [ ]:
prices_quantities="""
WITH a AS (
    SELECT 
        productid, SUM(orderqty) as quantity
    FROM 
        SalesOrderDetail
    GROUP BY 
        productid
),
b AS(    
    SELECT 
        p.productID, pc.name AS category, psc.name AS subcategory, p.listprice
    FROM 
        product AS p
    JOIN 
        productsubcategory AS psc ON psc.productsubcategoryID = p.productsubcategoryID
    JOIN 
        productcategory AS pc ON psc.productcategoryID = pc.productcategoryID
    GROUP BY 
        p.productID
)
SELECT 
    b.category, b.subcategory, AVG(b.listprice) AS average_price_in_subcategory, SUM(a.quantity) AS total_items_sold_in_subcategory
FROM 
    a
JOIN 
    b ON a.productID=b.productID
GROUP BY category, subcategory
"""


In [ ]:
%%sql 
WITH a AS (
    SELECT 
        productid, SUM(orderqty) as quantity
    FROM 
        SalesOrderDetail
    GROUP BY 
        productid
),
b AS(    
    SELECT 
        p.productID, pc.name AS category, psc.name AS subcategory, p.listprice
    FROM 
        product AS p
    JOIN 
        productsubcategory AS psc ON psc.productsubcategoryID = p.productsubcategoryID
    JOIN 
        productcategory AS pc ON psc.productcategoryID = pc.productcategoryID
    GROUP BY 
        p.productID
)
SELECT 
    b.category, b.subcategory, AVG(b.listprice) AS average_price_in_subcategory, SUM(a.quantity) AS total_items_sold_in_subcategory
FROM 
    a
JOIN 
    b ON a.productID=b.productID
GROUP BY category, subcategory


 * sqlite:///AdventureWorks.db
Done.


category,subcategory,average_price_in_subcategory,total_items_sold_in_subcategory
Accessories,Bike Racks,120.0,3166
Accessories,Bike Stands,159.0,249
Accessories,Bottles and Cages,7.989999999999999,10552
Accessories,Cleaners,7.95,3319
Accessories,Fenders,21.98,2121
Accessories,Helmets,34.99,19541
Accessories,Hydration Packs,54.99,2761
Accessories,Locks,25.0,1087
Accessories,Pumps,19.99,1130
Accessories,Tires and Tubes,19.482727272727274,18006


There is positive correlation between average price and items sold ($\rho=0.68$). This is somewhat unexpected, since common sense tells us that the more expensive an item is, the lower the demand for it. It is possible that we are witnessing an instance of Simpson's Paradox here. To verify if that is indeed the case, we could instead compute the correlation coefficient for each subcategory, possibly evidencing a negative correlation coefficient in some subcategories. We will not do that right now, however, since it would make us deviate too much from our business problem.

## Finding our top salespeople

As mentioned earlier, we want to find our best salespeople and see whether or not we can incentivize them in an appropriate manner. Namely, we want to determine if the commission percentage we give them motivates them to make more and bigger sales.

### Exercise 4 (1 point)

Find the top five performing salespeople by using the `salesytd` (Sales, year-to-date) column.

Your output should look like this:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>businessentityid</th>      <th>salesytd</th>    </tr>  </thead>  <tbody>    <tr>      <td>276</td>      <td>4251368.5497</td>    </tr>    <tr>      <td>289</td>      <td>4116871.2277</td>    </tr>    <tr>      <td>275</td>      <td>3763178.1787</td>    </tr>    <tr>      <td>...</td>      <td>...</td>    </tr>  </tbody></table>

**Hint:** We only need to know the `businessentityid` for each salesperson as this uniquely identifies each salesperson. Your query should therefore only have two columns: `businessentityid` and `salesytd`.

In [ ]:
salesperson_sales="""
SELECT 
    businessentityid, salesytd
FROM 
    SalesPerson
ORDER BY 
    salesytd desc
LIMIT 5
"""


In [ ]:
%%sql
SELECT *
FROM SalesPerson

 * sqlite:///AdventureWorks.db
Done.


businessentityid,territoryid,salesquota,bonus,commissionpct,salesytd,saleslastyear,rowguid,modifieddate
274,,,0,0.0,559697.5639,0.0,48754992-9ee0-4c0e-8c94-9451604e3e02,2010-12-28 00:00:00
275,2,300000,4100,0.012,3763178.1787,1750406.4785,1e0a7274-3064-4f58-88ee-4c6586c87169,2011-05-24 00:00:00
276,4,250000,2000,0.015,4251368.5497,1439156.0291,4dd9eee4-8e81-4f8c-af97-683394c1f7c0,2011-05-24 00:00:00
277,3,250000,2500,0.015,3189418.3662,1997186.2037,39012928-bfec-4242-874d-423162c3f567,2011-05-24 00:00:00
278,6,250000,500,0.01,1453719.4653,1620276.8966,7a0ae1ab-b283-40f9-91d1-167abf06d720,2011-05-24 00:00:00
279,5,300000,6700,0.01,2315185.611,1849640.9418,52a5179d-3239-4157-ae29-17e868296dc0,2011-05-24 00:00:00
280,1,250000,5000,0.01,1352577.1325,1927059.178,be941a4a-fb50-4947-bda4-bb8972365b08,2011-05-24 00:00:00
281,4,250000,3550,0.01,2458535.6169,2073505.9999,35326ddb-7278-4fef-b3ba-ea137b69094e,2011-05-24 00:00:00
282,6,250000,5000,0.015,2604540.7172,2038234.6549,31fd7fc1-dc84-4f05-b9a0-762519eacacc,2011-05-24 00:00:00
283,1,250000,3500,0.012,1573012.9383,1371635.3158,6bac15b2-8ffb-45a9-b6d5-040e16c2073f,2011-05-24 00:00:00


In [ ]:
%%sql
SELECT 
    businessentityid, salesytd
FROM 
    SalesPerson
ORDER BY 
    salesytd desc

 * sqlite:///AdventureWorks.db
Done.


businessentityid,salesytd
276,4251368.5497
289,4116871.2277
275,3763178.1787
277,3189418.3662
290,3121616.3202
282,2604540.7172
281,2458535.6169
279,2315185.611
288,1827066.7118
284,1576562.1966


### Exercise 5 (2 points)

The sales numbers from the previous query are hard-coded into the `salesperson` table, instead of dynamically calculated from each sales record. Currently, we don't know how this number is updated or much about it at all, so it's good to remain skeptical.

Using the ```salesorderheader``` table, find the top 5 salespeople who made the most sales *in the most recent year available* (2014). (There is a column called `subtotal` - use that.) Sales that do not have an associated salesperson should be excluded from your calculations and final output. All orders that were made within the 2014 calendar year should be included.

Your output should look like this:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>salespersonid</th>      <th>totalsales</th>    </tr>  </thead>  <tbody>    <tr>      <td>289</td>      <td>1382996.5839000002</td>    </tr>    <tr>      <td>276</td>      <td>1271088.5216</td>    </tr>    <tr>      <td>...</td>      <td>...</td>    </tr>  </tbody></table>

**Hint:** You can use the syntax `WHERE column >= '1970-01-01'` to generate an arbitrary date in SQLite and compare this to specific dates in the tables (in this example, dates equal to or later than Jan 1, 1970). Additionally, when you want to make sure that columns with empty or null values are excluded from a query in SQLite, you have to add a line like this one to your `WHERE` statement: `my_column IS NOT NULL AND my_column <> ""`. The `<>` operator is the opposite of `=`, that is, it checks that two values are different from each other.

In [ ]:
salesperson_totalsales="""
SELECT 
    salespersonid, SUM(subtotal) AS totalsales
FROM 
    salesorderheader
WHERE
    orderdate >= '2014-01-01' AND salespersonid IS NOT NULL AND salespersonid <> ""
GROUP BY
    salespersonid
ORDER BY
    totalsales DESC
LIMIT 5
"""


In [ ]:
%%sql


SELECT 
    salespersonid, SUM(subtotal) AS totalsales
FROM 
    salesorderheader
WHERE
    orderdate >= '2014-01-01' AND salespersonid IS NOT NULL AND salespersonid <> ""
GROUP BY
    salespersonid
ORDER BY
    totalsales DESC
LIMIT 5

 * sqlite:///AdventureWorks.db
Done.


salespersonid,totalsales
289,1382996.5839000002
276,1271088.5216
275,1057247.3786
282,1044810.8277000003
277,1040093.4071


You should see right away that there are discrepancies between the two sales totals. This makes sense because we used filters in one table and not the other. Nonetheless, for the remainder of this case, use this dynamically-calculated total as the authoritative answer.

### Exercise 6

Looking at the documentation, you will see that `subtotal` in the ```salesorderheader``` table is calculated from other tables in the database. To validate this figure (instead of trusting it blindly), it could be a good idea to calculate `subtotal` manually. Using the ```salesorderdetail``` and ```salesorderheader``` tables, let's calculate the sales for each salesperson for **the year 2014** and display results for the top 5 salespeople.

#### 6.1 (1 point)

Write a query that shows for each `salesorderid` (find this column in the `salesorderdetail` table) the total amount of money paid. Remember to subtract `unitpricediscount` from each item's price (`unitpricediscount` is a percentage).

Your output should look like this:

| salesorderid 	| ordertotal 	|
|-:	|-:	|
| 43659 	| 20565.6206 	|
| 43660 	| 1294.2529 	|
| 43661 	| 32726.4786 	|
| 43662 	| 28832.5289 	|
| 43663 	| 419.4589 	|
| 43664 	| 24432.608799999995 	|
| 43665 	| 14352.7713 	|
| 43666 	| 5056.4896 	|
| 43667 	| 6107.081999999999 	|
| 43668 	| 35944.156200000005 	|
| 43669 	| 714.7043 	|
| ... 	| ... 	|

In [ ]:
order_ordertotal="""
SELECT 
    salesorderid, sum(unitprice * (1 - unitpricediscount) * orderqty) AS ordertotal
FROM 
    salesorderdetail
GROUP BY 
    salesorderid
"""

#### 6.2 (2 points)

Using the previous query as a subquery, find the sales for each salesperson for the year 2014 and display results for the top 5 salespeople. Remember to exclude sales that are not associated with a salesperson.

**Hint:** You can get the `salesorderid` and `salespersonid` pairs from the `salesorderheader` table.

In [ ]:
salesperson_ordertotal="""
WITH a AS(
    SELECT 
        salesorderid, sum(unitprice * (1 - unitpricediscount) * orderqty) AS ordertotal
    FROM 
        salesorderdetail
    GROUP BY 
        salesorderid
)

SELECT 
    salespersonid, SUM(subtotal) AS ordertotalsum
FROM 
    salesorderheader
INNER JOIN
    a ON a.salesorderid = salesorderheader.salesorderid
WHERE
    orderdate >= '2014-01-01' AND salespersonid IS NOT NULL AND salespersonid <> ""
GROUP BY
    salespersonid
ORDER BY
    ordertotalsum DESC
LIMIT 5
"""


In [ ]:
%%sql

WITH a AS(
    SELECT 
        salesorderid, sum(unitprice * (1 - unitpricediscount) * orderqty) AS ordertotal
    FROM 
        salesorderdetail
    GROUP BY 
        salesorderid
)

SELECT 
    salespersonid, SUM(subtotal) AS ordertotalsum
FROM 
    salesorderheader
INNER JOIN
    a ON a.salesorderid = salesorderheader.salesorderid
WHERE
    orderdate >= '2014-01-01' AND salespersonid IS NOT NULL AND salespersonid <> ""
GROUP BY
    salespersonid
ORDER BY
    ordertotalsum DESC
LIMIT 5





 * sqlite:///AdventureWorks.db
Done.


salespersonid,ordertotalsum
289,1382996.5839000002
276,1271088.5216
275,1057247.3786
282,1044810.8277000003
277,1040093.4071


The results are the same as Exercise 5. We still prefer this query though because it is generated from granular data instead of relying on hard-coded figures.

### Exercise 7 (3 points)

Let's now see whether there is a positive relationship between the total sales of the salespeople and their commission percentages. Join the previous query (remove the `LIMIT` clause) with the `salesperson` table to get a table like this one:

| salespersonid 	| ordertotalsum 	| commissionpct 	|
|-:	|-:	|-:	|
| 274 	| 178584.36250800002 	| 0.0 	|
| 275 	| 1057247.378572 	| 0.012 	|
| 276 	| 1271088.5214610002 	| 0.015 	|
| 277 	| 1040093.406901 	| 0.015 	|
| ... 	| ... 	| ... 	|

**Hint:** Remember that the `businessentityid` column from the `salesperson` is compatible with the `salespersonid` column in the query of exercise 6 (they both represent the salesperson ID).

In [ ]:
salesperson_ordertotal_commission="""
WITH a AS(
    SELECT 
        salesorderid, sum(unitprice * (1 - unitpricediscount) * orderqty) AS ordertotal
    FROM 
        salesorderdetail
    GROUP BY 
        salesorderid
),
b AS(
    SELECT 
        businessentityid, commissionpct
    FROM
        salesperson
)

SELECT 
    salespersonid, SUM(subtotal) AS ordertotalsum, b.commissionpct
FROM 
    salesorderheader
INNER JOIN
    a ON a.salesorderid = salesorderheader.salesorderid
INNER JOIN
    b ON b.businessentityid = salesorderheader.salespersonid

WHERE
    orderdate >= '2014-01-01' AND salespersonid IS NOT NULL AND salespersonid <> ""
GROUP BY
    salespersonid
ORDER BY
    salespersonid
"""


In [ ]:
%%sql

WITH a AS(
    SELECT 
        salesorderid, sum(unitprice * (1 - unitpricediscount) * orderqty) AS ordertotal
    FROM 
        salesorderdetail
    GROUP BY 
        salesorderid
),
b AS(
    SELECT 
        businessentityid, commissionpct
    FROM
        salesperson
)

SELECT 
    salespersonid, SUM(subtotal) AS ordertotalsum, b.commissionpct
FROM 
    salesorderheader
INNER JOIN
    a ON a.salesorderid = salesorderheader.salesorderid
INNER JOIN
    b ON b.businessentityid = salesorderheader.salespersonid

WHERE
    orderdate >= '2014-01-01' AND salespersonid IS NOT NULL AND salespersonid <> ""
GROUP BY
    salespersonid
ORDER BY
    salespersonid

 * sqlite:///AdventureWorks.db
Done.


salespersonid,ordertotalsum,commissionpct
274,178584.3625,0.0
275,1057247.3786,0.012
276,1271088.5216,0.015
277,1040093.4071,0.015
278,435948.9550999999,0.01
279,787204.4288999998,0.01
280,504932.04399999994,0.01
281,777941.6518999999,0.01
282,1044810.8277000003,0.015
283,490466.319,0.012


The correlation coefficient between `ordertotalsum` and `commissionpct` is $\rho=0.73$. This suggests that the salespeople who earn a high commission are also those who close the bigger deals.

## Exercise 8 

Remember how we mentioned that products were sold in many regions? This is why you had to work with the `culture` value before to get the English language descriptions. To make matters worse, you are told the sales are recorded in *local* currency, so your previous analyses are flawed. Technically, you must convert all amounts to USD if you wish to compare the different salespeople fairly! Instead, let's group the salespeople orders by the currency used for each order (you will have to consider `tocurrencyrate` for this task in the `CurrencyRate` table). 

Let's explore the currencies in different sales. But first, here are some things to understand about the currency columns:
* The `FromCurrencyCode` is all USD, so focus on `tocurrencyrate`
* If the sale was paid in USD, the `currencyrateid` was left blank (since there was no need to make a conversion)

#### 8.1 (1 point)

Create a table with the `salespersonid`, `salesorderid`, `currencyrateid` and `tocurrencycode` to see the connection. Remember to exclude sales that are not associated with a salesperson and only consider sales in 2014. Order by the salesperson ID and show only 10 rows. Your table should look like this

| salespersonid 	| salesorderid	| currencyrateid	|tocurrencycode   |
|-:	|-:	|-:	|-: |
| 274 	| 65294 	| None 	| None |
| 274 	| 65298 	| None 	| None |
| 274 	| 67277 	| None 	| None |
| 274 	| 67286	 	| 11427	 	| CAD |
| 274 	| 69528		 	| None	| None |
| ... 	| ... 	| ... 	| ...    |

**Hint**: Since `USD` would not show up in the **CurrencyRate** table, you will have to do a `LEFT JOIN` to avoid losing information. 

In [ ]:
salesperson_currency_id="""
WITH a AS(
    SELECT 
        currencyrateid, tocurrencycode
    FROM 
        currencyrate
)
SELECT 
    salespersonid, salesorderid, a.currencyrateid, a.tocurrencycode
FROM 
    salesorderheader
LEFT JOIN
    a ON a.currencyrateid = salesorderheader.currencyrateid
WHERE
    orderdate >= '2014-01-01' AND salespersonid IS NOT NULL AND salespersonid <> ""
ORDER BY
    salespersonid
LIMIT 10
"""


In [ ]:
%%sql

SELECT *
FROM salesorderheader
LIMIT 10

 * sqlite:///AdventureWorks.db
Done.


salesorderid,revisionnumber,orderdate,duedate,shipdate,STATUS,onlineorderflag,purchaseordernumber,accountnumber,customerid,salespersonid,territoryid,billtoaddressid,shiptoaddressid,shipmethodid,creditcardid,creditcardapprovalcode,currencyrateid,subtotal,taxamt,freight,totaldue,comment,rowguid,modifieddate
43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO522145787,10-4020-000676,29825,279,5,985,985,5,16281,105041Vi84182,,20565.6206,1971.5149,616.0984,23153.2339,,79b65321-39ca-4115-9cba-8fe0903e12e6,2011-06-07 00:00:00
43660,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO18850127500,10-4020-000117,29672,279,5,921,921,5,5618,115213Vi29411,,1294.2529,124.2483,38.8276,1457.3288,,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07 00:00:00
43661,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO18473189620,10-4020-000442,29734,282,6,517,517,5,1346,85274Vi6854,4,32726.4786,3153.7696,985.553,36865.8012,,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07 00:00:00
43662,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO18444174044,10-4020-000227,29994,282,6,482,482,5,10456,125295Vi53935,4,28832.5289,2775.1646,867.2389,32474.9324,,4a1ecfc0-cc3a-4740-b028-1c50bb48711c,2011-06-07 00:00:00
43663,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO18009186470,10-4020-000510,29565,276,4,1073,1073,5,4322,45303Vi22691,,419.4589,40.2681,12.5838,472.3108,,9b1e7a40-6ae0-4ad3-811c-a64951857c4b,2011-06-07 00:00:00
43664,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO16617121983,10-4020-000397,29898,280,1,876,876,5,806,95555Vi4081,,24432.6088,2344.9921,732.81,27510.4109,,22a8a5da-8c22-42ad-9241-839489b6ef0d,2011-06-07 00:00:00
43665,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO16588191572,10-4020-000146,29580,283,1,849,849,5,15232,35568Vi78804,,14352.7713,1375.9427,429.9821,16158.6961,,5602c304-853c-43d7-9e79-76e320d476cf,2011-06-07 00:00:00
43666,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO16008173883,10-4020-000511,30052,276,4,1074,1074,5,13349,105623Vi69217,,5056.4896,486.3747,151.9921,5694.8564,,e2a90057-1366-4487-8a7e-8085845ff770,2011-06-07 00:00:00
43667,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO15428132599,10-4020-000646,29974,277,3,629,629,5,10370,55680Vi53503,,6107.082,586.1203,183.1626,6876.3649,,86d5237d-432d-4b21-8abc-671942f5789d,2011-06-07 00:00:00
43668,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,f,PO14732180295,10-4020-000514,29614,282,6,529,529,5,1566,85817Vi8045,4,35944.1562,3461.7654,1081.8017,40487.7233,,281cc355-d538-494e-9b44-461b36a826c6,2011-06-07 00:00:00


In [ ]:
%%sql

WITH a AS(
    SELECT 
        currencyrateid, tocurrencycode
    FROM 
        currencyrate
)
SELECT 
    salespersonid, salesorderid, a.currencyrateid, a.tocurrencycode
FROM 
    salesorderheader
LEFT JOIN
    a ON a.currencyrateid = salesorderheader.currencyrateid
WHERE
    orderdate >= '2014-01-01' AND salespersonid IS NOT NULL AND salespersonid <> ""
ORDER BY
    salespersonid
LIMIT 10

 * sqlite:///AdventureWorks.db
Done.


salespersonid,salesorderid,currencyrateid,tocurrencycode
274,65294,None,None
274,65298,None,None
274,67277,None,None
274,67286,11427,CAD
274,69528,None,None
274,69545,None,None
274,71779,None,None
274,71919,None,None
275,65205,None,None
275,65215,None,None


As expected, we can see that different salespeople have sales in different currencies. 

**Note**: The `None` in the above example takes the place of `NULL` values, which contextually means that the sale was in USD. 

#### 8.2 (2 points)

The `None` in the above query can be confusing to someone who doesn't understand the database. In this case, it's best to replace them with useful information. Redo the previous exercise with the following changes:
* Leave out the `currencyrateid` column 
* Replace `None` with 'USD' in the `tocurrencycode` column

**Hint**: One way of completing this task is to use the `CASE` expression, which can be incorporated like this:

~~~sql
SELECT column1, column2, 
CASE
    WHEN condition1 THEN result1
    ELSE result2
END AS column3
FROM Table
~~~

The above would result in a table with the following columns

|column1   |column2   |column3   |
|-:  |-:   |-:   |
|...  |... |... |

In the `tocurrencycode` column, the `CASE` would 
* replace `NULL` values with 'USD'
* leave other values as they are  

In [ ]:
salesperson_currency_code="""
WITH a AS(
    SELECT 
        currencyrateid, tocurrencycode
    FROM 
        currencyrate
)
SELECT 
    salespersonid, salesorderid,
CASE WHEN 
    (tocurrencycode IS NULL) THEN 'USD'
ELSE
    tocurrencycode
END AS
    tocurrencycode
FROM 
    salesorderheader
LEFT JOIN
    a ON a.currencyrateid = salesorderheader.currencyrateid
WHERE
    orderdate >= '2014-01-01' AND salespersonid IS NOT NULL AND salespersonid <> ""
ORDER BY
    salespersonid
LIMIT 10
"""


In [ ]:
%%sql

WITH a AS(
    SELECT 
        currencyrateid, tocurrencycode
    FROM 
        currencyrate
)
SELECT 
    salespersonid, salesorderid,
CASE WHEN 
    (tocurrencycode IS NULL) THEN 'USD'
ELSE
    tocurrencycode
END AS
    tocurrencycode
FROM 
    salesorderheader
LEFT JOIN
    a ON a.currencyrateid = salesorderheader.currencyrateid
WHERE
    orderdate >= '2014-01-01' AND salespersonid IS NOT NULL AND salespersonid <> ""
ORDER BY
    salespersonid
LIMIT 10

 * sqlite:///AdventureWorks.db
Done.


salespersonid,salesorderid,tocurrencycode
274,65294,USD
274,65298,USD
274,67277,USD
274,67286,CAD
274,69528,USD
274,69545,USD
274,71779,USD
274,71919,USD
275,65205,USD
275,65215,USD


### Exercise 9 (3 points)

Now that we have the currency codes associated with each salesperson ID, redo Exercise 7 adding in the `tocurrencycode`. Order the results by currency (ascending) and total sales (descending) to make it easier to see who the best salespeople are for each currency.

**Hint:** Start with Exercise 7 and integrate the currency piece using the `CASE` expression and `LEFT JOIN` from Exercise 8.2 (removing the `LIMIT` clause).

In [ ]:
salesperson_ranking_currency="""
SELECT
   salespersonid, sum(unitprice * (1 - unitpricediscount) * orderqty) AS ordertotalsum, commissionpct,
CASE WHEN 
    (tocurrencycode IS NULL) THEN 'USD'
ELSE
    tocurrencycode
END AS
    tocurrencycode
FROM
    salesperson
INNER JOIN
    salesorderheader ON salesperson.businessentityid = salesorderheader.salespersonid
INNER JOIN
    salesorderdetail ON salesorderheader.salesorderid = salesorderdetail.salesorderid
LEFT JOIN
    currencyrate ON salesorderheader.currencyrateid = currencyrate.currencyrateid
WHERE
    orderdate >= '2014-01-01' AND salespersonid is not NULL AND salespersonid <> ""
GROUP BY
    salespersonid, tocurrencycode, commissionpct
ORDER BY
    tocurrencycode, ordertotalsum DESC
"""


In [ ]:
%%sql

SELECT
   salespersonid, sum(unitprice * (1 - unitpricediscount) * orderqty) AS ordertotalsum, commissionpct,
CASE WHEN 
    (tocurrencycode IS NULL) THEN 'USD'
ELSE
    tocurrencycode
END AS
    tocurrencycode
FROM
    salesperson
INNER JOIN
    salesorderheader ON salesperson.businessentityid = salesorderheader.salespersonid
INNER JOIN
    salesorderdetail ON salesorderheader.salesorderid = salesorderdetail.salesorderid
LEFT JOIN
    currencyrate ON salesorderheader.currencyrateid = currencyrate.currencyrateid
WHERE
    orderdate >= '2014-01-01' AND salespersonid is not NULL AND salespersonid <> ""
GROUP BY
    salespersonid, tocurrencycode, commissionpct
ORDER BY
    tocurrencycode, ordertotalsum DESC

 * sqlite:///AdventureWorks.db
Done.


salespersonid,ordertotalsum,commissionpct,tocurrencycode
286,585755.8005279993,0.018,AUD
285,21267.336,0.0,AUD
289,1382996.583909995,0.02,CAD
278,435948.9551659999,0.01,CAD
274,11802.563999999998,0.0,CAD
288,581358.3893809991,0.018,EUR
290,88188.92707200002,0.016,EUR
287,55965.4538,0.0,EUR
282,1044810.8276869993,0.015,GBP
287,672.294,0.0,GBP


## Testing cells

In [ ]:
import sqlalchemy
sqlite_engine = sqlalchemy.create_engine("sqlite:///AdventureWorks.db")

In [ ]:
# Ex. 1
assert "rating_ranking" in globals(), "Ex. 1 - Remember that your variable's name should be `rating_ranking`!"
rating_ranking_result = pd.read_sql(rating_ranking, con=sqlite_engine)
assert len(rating_ranking_result) > 0, "Ex. 1 - Your code is not producing any output! (ie., a table with lenght zero)"
assert set(rating_ranking_result.columns) == {'NAME', 'avgrating', 'num_ratings', 'productid'}, "Ex. 1 - Your query result doesn't have exactly these columns: 'NAME', 'avgrating', 'num_ratings', 'productid'"
print("Exercise 1 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 1 looks fine for now. You will get your final grade after we've reviewed your submission.


In [ ]:
# Ex. 2.1
assert "productmodelid_description" in globals(), "Ex. 2.1 - Remember that your variable's name should be `productmodelid_description`!"
productmodelid_description_result = pd.read_sql(productmodelid_description, con=sqlite_engine)
assert len(productmodelid_description_result) == 127, "Ex. 2.1 - There are 127 product models in the database, but your query produces a different number. Make sure that you don't have any LIMIT clauses in this exercise!"
assert set(productmodelid_description_result.columns) == {'description', 'productmodelid'}, "Ex. 2.1 - Your query result doesn't have exactly these columns: 'description', 'productmodelid'"
print("Exercise 2.1 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 2.1 looks fine for now. You will get your final grade after we've reviewed your submission.


In [ ]:
# Ex. 2.2
assert "description_totalorders" in globals(), "Ex. 2.2 - Remember that your variable's name should be `description_totalorders`!"
description_totalorders_result = pd.read_sql(description_totalorders, con=sqlite_engine)
assert len(description_totalorders_result) == 10, "Ex. 2.2 - Remember to use LIMIT 10 in your query! This is a top 10!"
assert set(description_totalorders_result.columns) == {'NAME', 'description', 'productmodelid', 'total_orders'}, "Ex. 2.2 - Your query result doesn't have exactly these columns: 'NAME', 'description', 'productmodelid', 'total_orders'"
print("Exercise 2.2 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 2.2 looks fine for now. You will get your final grade after we've reviewed your submission.


In [ ]:
# Ex. 3.1
assert "quantities_ordered" in globals(), "Ex. 3.1 - Remember that your variable's name should be `quantities_ordered`!"
quantities_ordered_result = pd.read_sql(quantities_ordered, con=sqlite_engine)
assert len(quantities_ordered_result) == 266, "Ex. 2 - There are 266 products in the database that have associated quantities, but your query produces a different number. Make sure that you don't have any LIMIT clauses in this exercise and don't filter by culture!"
assert set(quantities_ordered_result.columns) == {'productid', 'quantity'}, "Ex. 3.1 - Your query result doesn't have exactly these columns: 'productid', 'quantity'"
print("Exercise 3.1 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 3.1 looks fine for now. You will get your final grade after we've reviewed your submission.


In [ ]:
# Ex. 3.2
assert "products_prices" in globals(), "Ex. 3.2 - Remember that your variable's name should be `products_prices`!"
products_prices_result = pd.read_sql(products_prices, con=sqlite_engine)
assert len(products_prices_result) == 295, "Ex. 3.2 - There are 295 products in the database that have prices, but your query produces a different number. Make sure that you don't have any LIMIT clauses in this exercise!"
assert set(products_prices_result.columns) == {'category', 'listprice', 'productid', 'subcategory'}, "Ex. 3.2 - Your query result doesn't have exactly these columns: 'category', 'listprice', 'productid', 'subcategory'"
print("Exercise 3.2 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 3.2 looks fine for now. You will get your final grade after we've reviewed your submission.


In [ ]:
# Ex. 3.3
assert "prices_quantities" in globals(), "Ex. 3.3 - Remember that your variable's name should be `prices_quantities`!"
prices_quantities_result = pd.read_sql(prices_quantities, con=sqlite_engine)
assert len(prices_quantities_result) == 35, "Ex. 3.3 - There are 35 subcategories in the database, but your query produces a different number. Make sure that you don't have any LIMIT clauses in this exercise!"
assert set(prices_quantities_result.columns) == {'average_price_in_subcategory', 'category', 'subcategory', 'total_items_sold_in_subcategory'}, "Ex. 3.3 - Your query result doesn't have exactly these columns: 'average_price_in_subcategory', 'category', 'subcategory', 'total_items_sold_in_subcategory'"
print("Exercise 3.3 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 3.3 looks fine for now. You will get your final grade after we've reviewed your submission.


In [ ]:
# Ex. 4
assert "salesperson_sales" in globals(), "Ex. 4 - Remember that your variable's name should be `salesperson_sales`!"
salesperson_sales_result = pd.read_sql(salesperson_sales, con=sqlite_engine)
assert len(salesperson_sales_result) == 5, "Ex. 4 - This is a top 5. Remember to use LIMIT!"
assert set(salesperson_sales_result.columns) == {'businessentityid', 'salesytd'}, "Ex. 4 - Your query result doesn't have exactly these columns: 'businessentityid', 'salesytd'"
print("Exercise 4 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 4 looks fine for now. You will get your final grade after we've reviewed your submission.


In [ ]:
# Ex. 5
assert "salesperson_totalsales" in globals(), "Ex. 5 - Remember that your variable's name should be `salesperson_totalsales`!"
salesperson_totalsales_result = pd.read_sql(salesperson_totalsales, con=sqlite_engine)
assert len(salesperson_totalsales_result) == 5, "Ex. 5 - This is a top 5. Remember to use LIMIT!"
assert set(salesperson_totalsales_result.columns) == {'salespersonid', 'totalsales'}, "Ex. 5 - Your query result doesn't have exactly these columns: 'salespersonid', 'totalsales'"
print("Exercise 5 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 5 looks fine for now. You will get your final grade after we've reviewed your submission.


In [ ]:
# Ex. 6.1
assert "order_ordertotal" in globals(), "Ex. 6.1 - Remember that your variable's name should be `order_ordertotal`!"
order_ordertotal_result = pd.read_sql(order_ordertotal, con=sqlite_engine)
assert len(order_ordertotal_result) == 31465, "Ex. 6.1 - There are more than 31,000 orders in the database. Remember to NOT use LIMIT here!"
assert set(order_ordertotal_result.columns) == {'ordertotal', 'salesorderid'}, "Ex. 6.1 - Your query result doesn't have exactly these columns: 'ordertotal', 'salesorderid'"
print("Exercise 6.1 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 6.1 looks fine for now. You will get your final grade after we've reviewed your submission.


In [ ]:
# Ex. 6.2
assert "salesperson_ordertotal" in globals(), "Ex. 6.2 - Remember that your variable's name should be `salesperson_ordertotal`!"
salesperson_ordertotal_result = pd.read_sql(salesperson_ordertotal, con=sqlite_engine)
assert len(salesperson_ordertotal_result) == 5, "Ex. 6.2 - There are too many or too few rows in your result. Remember to use LIMIT here!"
assert set(salesperson_ordertotal_result.columns) == {'ordertotalsum', 'salespersonid'}, "Ex. 6.2 - Your query result doesn't have exactly these columns: 'ordertotalsum', 'salespersonid'"
print("Exercise 6.2 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 6.2 looks fine for now. You will get your final grade after we've reviewed your submission.


In [ ]:
# Ex. 7
assert "salesperson_ordertotal_commission" in globals(), "Ex. 7 - Remember that your variable's name should be `salesperson_ordertotal_commission`!"
salesperson_ordertotal_commission_result = pd.read_sql(salesperson_ordertotal_commission, con=sqlite_engine)
assert len(salesperson_ordertotal_commission_result) == 17, "Ex. 7 - There are too many or too few rows in your result. Remember to NOT use LIMIT here!"
assert set(salesperson_ordertotal_commission_result.columns) == {'commissionpct', 'ordertotalsum', 'salespersonid'}, "Ex. 7 - Your query result doesn't have exactly these columns: 'commissionpct', 'ordertotalsum', 'salespersonid'"
print("Exercise 7 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 7 looks fine for now. You will get your final grade after we've reviewed your submission.


In [ ]:
# Ex. 8.1
assert "salesperson_currency_id" in globals(), "Ex. 8.1 - Remember that your variable's name should be `salesperson_currency_id`!"
salesperson_currency_result = pd.read_sql(salesperson_currency_id, con=sqlite_engine)
assert len(salesperson_currency_result) == 10, "Ex. 8.1 - There are too many or too few rows in your result. Remember to use LIMIT here!"
assert set(salesperson_currency_result.columns) == {'salespersonid', 'salesorderid', 'currencyrateid','tocurrencycode'}, "Ex. 8.1 - Your query result doesn't have exactly these columns: 'salespersonid', 'salesorderid', 'currencyrateid','tocurrencycode'"
print("Exercise 8.1 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 8.1 looks fine for now. You will get your final grade after we've reviewed your submission.


In [ ]:
# Ex. 8.2
assert "salesperson_currency_code" in globals(), "Ex. 8.2 - Remember that your variable's name should be `salesperson_currency_code`!"
salesperson_currency_result2 = pd.read_sql(salesperson_currency_code, con=sqlite_engine)
assert len(salesperson_currency_result2) == 10, "Ex. 8.2 - There are too many or too few rows in your result. Remember to use LIMIT here!"
assert set(salesperson_currency_result2.columns) == {'salespersonid', 'salesorderid','tocurrencycode'}, "Ex. 8.2 - Your query result doesn't have exactly these columns: 'salespersonid', 'salesorderid', 'currencyrateid','tocurrencycode'"
print("Exercise 8.2 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 8.2 looks fine for now. You will get your final grade after we've reviewed your submission.


In [ ]:
# Ex. 9
assert "salesperson_ranking_currency" in globals(), "Ex. 9 - Remember that your variable's name should be `salesperson_ranking_currency`!"
salesperson_ranking_currency_result = pd.read_sql(salesperson_ranking_currency, con=sqlite_engine)
assert len(salesperson_ranking_currency_result) == 21, "Ex. 9 - There are too many or too few rows in your result. Remember to NOT use LIMIT here!"
assert set(salesperson_ranking_currency_result.columns) == {'salespersonid', 'tocurrencycode', 'ordertotalsum', 'commissionpct'}, "Ex. 9 - Your query result doesn't have exactly these columns: 'salespersonid', 'tocurrencycode', 'ordertotalsum', 'commissionpct'"
print("Exercise 9 looks fine for now. You will get your final grade after we've reviewed your submission.")

Exercise 9 looks fine for now. You will get your final grade after we've reviewed your submission.


## Attribution

"AdventureWorks database", Nov 7, 2017, Microsoft Corporation, [MIT License](https://docs.microsoft.com/en-us/sql/samples/sql-samples-where-are?view=sql-server-ver15), https://github.com/microsoft/sql-server-samples/tree/master/samples/databases/adventure-works